In [ ]:
from pathlib import Path
import cv2

def find_all_jpg_files(directory):
    return [str(f) for f in Path(directory).rglob("*.JPG")]

def get_qr_code_data(decoder, image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_qr = image[100:400, 2050:2350]
    data, points = decoder.detectAndDecode(image_qr)
    try:
        return data[0]
    except:
        return None

def get_left_top_number(image_path, templates):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 假设左上角数字的位置在[50:200, 50:300]
    image_number = image[50:200, 50:300]
    
    detected_number = ""
    
    # 遍历每一个模板
    for digit, template in templates.items():
        result = cv2.matchTemplate(image_number, template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(result)
        
        # 如果匹配度足够高，则认为数字被识别
        if max_val > 0.95:  # 你可能需要调整这个阈值
            detected_number += str(digit)
    
    return detected_number

# 加载数字模板
templates = {
    0: cv2.imread('0.png', 0),
    1: cv2.imread('1.png', 0),
    2: cv2.imread('2.png', 0),
    3: cv2.imread('3.png', 0),
    4: cv2.imread('4.png', 0),
    5: cv2.imread('5.png', 0),
    6: cv2.imread('6.png', 0),
    7: cv2.imread('7.png', 0),
    8: cv2.imread('8.png', 0),
    9: cv2.imread('9.png', 0),
}

image_paths = find_all_jpg_files("231005")
results = []

decoder = cv2.wechat_qrcode_WeChatQRCode()

for image_path in image_paths:
    result = {}
    result["image_path"] = image_path
    result["qr_data"] = get_qr_code_data(decoder, image_path)
    result["left_top_number"] = get_left_top_number(image_path, templates)
    results.append(result)

replacement_values = ['ggvpfLW4ylL7GCpHTxLfa656', 'vc5BgBkSGb5blQ2pbTYYCGsi']
replacement_index = 0

for i, result in enumerate(results):
    if result["qr_data"] is None:
        result["qr_data"] = replacement_values[replacement_index]
        replacement_index += 1

all_data = [result["qr_data"] for result in results]

duplicate_results = [result for result in results if all_data.count(result["qr_data"]) > 1]

if not duplicate_results:
    print("没有人copy")
else:
    print(duplicate_results)
